<a href="https://colab.research.google.com/github/SQLTECHPRO/SQL/blob/main/google_casual_impact.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Libraries and Dates

In [ ]:
#install libraries
!pip install yfinance
!pip install tfcausalimpact

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.2/59.2 KB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 KB 6.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
  Attempting uninstall: html5lib
    Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
   

In [ ]:
#import libraries
import yfinance as yf
from causalimpact import CausalImpact
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
#Dates
training_start = '2020-09-01'
training_end ='2020-10-19'
treatment_start = '2020-10-20'
treatment_end = '2020-10-23'
end_stock = '2020-10-24'


#loading financial data

In [ ]:
#Bitcoin
y = ['BTC-USD']
y = yf.download(tickers = y,
                start = training_start,
                end = end_stock,
                interval = '1d')
y = y['Adj Close'].rename('y')
y[:1]

[*********************100%***********************]  1 of 1 completed


Date
2020-09-01    11970.478516
Name: y, dtype: float64

In [ ]:
#Load more data
stocks = ["ZAL.De", "SQ", "CRSP", "TRMB", "JD", "DE", "KTOS", "GOOG"]
X = yf.download(tickers = stocks,
                start = training_start,
                end = end_stock,
                interval = "1d")
print(X)

[*********************100%***********************]  8 of 8 completed
             Adj Close                         ...      Volume                    
                  CRSP          DE       GOOG  ...          SQ       TRMB   ZAL.DE
Date                                           ...                                
2020-09-01   93.419998  211.338715  83.035500  ...  12306400.0   542300.0   790198
2020-09-02   93.930000  210.775635  86.414001  ...  11214800.0   728800.0   496554
2020-09-03   85.690002  204.659439  82.092003  ...  16421200.0  1220300.0   736259
2020-09-04   82.019997  205.173981  79.552002  ...  17995200.0   914700.0   662250
2020-09-07         NaN         NaN        NaN  ...         NaN        NaN   362062
2020-09-08   81.459999  204.106094  76.619499  ...  10681400.0  1007900.0   494693
2020-09-09   85.250000  209.232025  77.848000  ...   9463600.0   881000.0   477716
2020-09-10   81.800003  208.833984  76.600998  ...   6711500.0   783500.0   432871
2020-09-11   79.98

#Data Preparation

In [ ]:
#Getting the adjusted closed
X = X.iloc[:, :len(stocks)]
X.head(5)

,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE
Date,,,,,,,,
2020-09-01,93.419998,211.338730,83.035500,80.459969,19.700001,166.660004,53.410000,77.000000
2020-09-02,93.930000,210.775635,86.414001,81.162254,19.910000,162.880005,54.310001,77.000000
2020-09-03,85.690002,204.659454,82.092003,77.094872,19.520000,152.860001,50.900002,75.080002
2020-09-04,82.019997,205.173996,79.552002,78.031250,19.260000,146.389999,49.959999,71.739998
2020-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.000000


In [ ]:
#Removing level
X.columns = X.columns.droplevel()
X.head()

,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-09-01,93.419998,211.338715,83.035500,80.459969,19.700001,166.660004,53.410000,77.000000,93.419998,217.690002,83.035500,82.489998,19.700001,166.660004,53.410000,77.000000,94.500000,217.720001,83.286499,83.000000,19.93,170.460007,53.470001,77.500000,90.730003,208.270004,81.611000,79.144997,19.230000,162.000000,52.099998,74.779999,91.989998,208.520004,81.831497,79.489998,19.459999,164.809998,52.669998,74.959999,779500.0,1873400.0,36506000.0,11431400.0,587600.0,12306400.0,542300.0,790198
2020-09-02,93.930000,210.775635,86.414001,81.162254,19.910000,162.880005,54.310001,77.000000,93.930000,217.110001,86.414001,83.209999,19.910000,162.880005,54.310001,77.000000,94.410004,219.710007,86.658997,86.580002,20.00,170.610001,54.509998,78.080002,90.714996,215.669998,83.316498,81.800003,19.580000,158.110001,52.959999,76.220001,94.150002,218.240005,83.688751,85.459000,19.730000,170.600006,53.779999,77.500000,532000.0,1983000.0,50224000.0,13860900.0,612800.0,11214800.0,728800.0,496554
2020-09-03,85.690002,204.659439,82.092003,77.094872,19.520000,152.860001,50.900002,75.080002,85.690002,210.809998,82.092003,79.040001,19.520000,152.860001,50.900002,75.080002,92.128998,218.729996,85.485703,81.690002,20.15,157.229996,54.130001,77.940002,84.195000,209.070007,80.752998,76.029999,19.299999,149.509995,50.660000,74.400002,91.690002,217.270004,85.485703,81.375000,20.000000,157.000000,54.130001,77.260002,1278900.0,1975200.0,62156000.0,19254000.0,847100.0,16421200.0,1220300.0,736259
2020-09-04,82.019997,205.173981,79.552002,78.031250,19.260000,146.389999,49.959999,71.739998,82.019997,211.339996,79.552002,80.000000,19.260000,146.389999,49.959999,71.739998,87.000000,214.000000,82.255501,80.800003,19.83,152.220001,51.529999,76.300003,76.709999,208.179993,77.380653,75.400002,18.840000,134.000000,49.070000,71.300003,86.489998,213.190002,81.212997,77.966003,19.680000,149.630005,51.040001,74.800003,1570300.0,1662300.0,52172000.0,21500900.0,1060000.0,17995200.0,914700.0,662250
2020-09-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,74.019997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,71.940002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.239998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,362062


In [ ]:
#Time Zones / no time zone
X.index = X.index.tz_localize(None)
X.index

DatetimeIndex(['2020-09-01', '2020-09-02', '2020-09-03', '2020-09-04',
               '2020-09-07', '2020-09-08', '2020-09-09', '2020-09-10',
               '2020-09-11', '2020-09-14', '2020-09-15', '2020-09-16',
               '2020-09-17', '2020-09-18', '2020-09-21', '2020-09-22',
               '2020-09-23', '2020-09-24', '2020-09-25', '2020-09-28',
               '2020-09-29', '2020-09-30', '2020-10-01', '2020-10-02',
               '2020-10-05', '2020-10-06', '2020-10-07', '2020-10-08',
               '2020-10-09', '2020-10-12', '2020-10-13', '2020-10-14',
               '2020-10-15', '2020-10-16', '2020-10-19', '2020-10-20',
               '2020-10-21', '2020-10-22', '2020-10-23'],
              dtype='datetime64[ns]', name='Date', freq='B')

In [ ]:
#Combine everything
X.index = X.index.tz_localize(None)
df = pd.concat([y, X], axis = 1).dropna()
df.head()

,y,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-09-01,11970.478516,93.419998,211.338715,83.035500,80.459969,19.700001,166.660004,53.410000,77.000000,93.419998,217.690002,83.035500,82.489998,19.700001,166.660004,53.410000,77.000000,94.500000,217.720001,83.286499,83.000000,19.93,170.460007,53.470001,77.500000,90.730003,208.270004,81.611000,79.144997,19.230000,162.000000,52.099998,74.779999,91.989998,208.520004,81.831497,79.489998,19.459999,164.809998,52.669998,74.959999,779500.0,1873400.0,36506000.0,11431400.0,587600.0,12306400.0,542300.0,790198.0
2020-09-02,11414.034180,93.930000,210.775635,86.414001,81.162254,19.910000,162.880005,54.310001,77.000000,93.930000,217.110001,86.414001,83.209999,19.910000,162.880005,54.310001,77.000000,94.410004,219.710007,86.658997,86.580002,20.00,170.610001,54.509998,78.080002,90.714996,215.669998,83.316498,81.800003,19.580000,158.110001,52.959999,76.220001,94.150002,218.240005,83.688751,85.459000,19.730000,170.600006,53.779999,77.500000,532000.0,1983000.0,50224000.0,13860900.0,612800.0,11214800.0,728800.0,496554.0
2020-09-03,10245.296875,85.690002,204.659439,82.092003,77.094872,19.520000,152.860001,50.900002,75.080002,85.690002,210.809998,82.092003,79.040001,19.520000,152.860001,50.900002,75.080002,92.128998,218.729996,85.485703,81.690002,20.15,157.229996,54.130001,77.940002,84.195000,209.070007,80.752998,76.029999,19.299999,149.509995,50.660000,74.400002,91.690002,217.270004,85.485703,81.375000,20.000000,157.000000,54.130001,77.260002,1278900.0,1975200.0,62156000.0,19254000.0,847100.0,16421200.0,1220300.0,736259.0
2020-09-04,10511.813477,82.019997,205.173981,79.552002,78.031250,19.260000,146.389999,49.959999,71.739998,82.019997,211.339996,79.552002,80.000000,19.260000,146.389999,49.959999,71.739998,87.000000,214.000000,82.255501,80.800003,19.83,152.220001,51.529999,76.300003,76.709999,208.179993,77.380653,75.400002,18.840000,134.000000,49.070000,71.300003,86.489998,213.190002,81.212997,77.966003,19.680000,149.630005,51.040001,74.800003,1570300.0,1662300.0,52172000.0,21500900.0,1060000.0,17995200.0,914700.0,662250.0
2020-09-08,10131.516602,81.459999,204.106094,76.619499,74.344276,19.230000,139.110001,49.139999,73.440002,81.459999,210.240005,76.619499,76.220001,19.230000,139.110001,49.139999,73.440002,84.139000,213.240005,78.193253,78.014999,19.52,146.139999,50.009998,74.320000,78.510002,207.970001,76.400497,75.379997,18.860001,136.889999,48.509998,71.239998,81.800003,209.860001,76.675499,76.000000,19.090000,137.009995,49.020000,74.180000,636900.0,1918500.0,52218000.0,14080500.0,1390300.0,10681400.0,1007900.0,494693.0


#Data Analysis Section




In [ ]:
#subsetting just the training data
df_training = df[df.index <= training_end]
df_training.tail(5)

,y,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE,CRSP,DE,GOOG,JD,KTOS,SQ,TRMB,ZAL.DE
Date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2020-10-13,11425.899414,103.059998,231.090378,78.584000,80.577019,20.959999,190.470001,51.720001,85.820000,103.059998,237.220001,78.584000,82.610001,20.959999,190.470001,51.720001,85.820000,103.400002,237.580002,79.500000,84.345001,21.209999,191.360001,52.639999,86.000000,97.089996,234.490005,78.160004,82.129997,20.770000,186.000000,51.639999,83.940002,100.010002,236.139999,79.186501,84.099998,20.980000,186.809998,52.000000,84.839996,1369600.0,1311300.0,32020000.0,12402300.0,682600.0,6399900.0,725500.0,458541.0
2020-10-14,11429.506836,101.370003,232.308075,78.403999,79.104179,21.010000,187.490005,50.540001,84.980003,101.370003,238.470001,78.403999,81.099998,21.010000,187.490005,50.540001,84.980003,110.719002,240.660004,79.384201,83.139999,21.410000,193.440002,52.349998,86.180000,100.750000,237.089996,77.526497,79.910004,20.940001,185.110001,50.509998,84.760002,103.915001,237.389999,78.929497,82.809998,21.010000,191.589996,51.889999,85.680000,2229100.0,1062500.0,38586000.0,12398300.0,572800.0,6165800.0,546000.0,638549.0
2020-10-15,11495.349609,99.889999,232.084015,77.956497,78.275093,21.100000,188.600006,52.290001,84.419998,99.889999,238.240005,77.956497,80.250000,21.100000,188.600006,52.290001,84.419998,101.650002,239.350006,78.755249,80.320000,21.170000,188.679993,52.590000,84.860001,98.010002,234.369995,77.251503,78.639999,20.610001,181.199997,49.580002,82.919998,99.389999,234.660004,77.357498,79.000000,20.709999,182.600006,50.060001,84.580002,956600.0,1225200.0,30800000.0,10540000.0,462400.0,6810100.0,1398900.0,463238.0
2020-10-16,11322.123047,109.830002,233.856995,78.650497,79.513840,20.790001,186.350006,52.340000,84.800003,109.830002,240.059998,78.650497,81.519997,20.790001,186.350006,52.340000,84.800003,111.900002,243.399994,79.056503,82.239998,21.270000,192.839996,53.290001,85.860001,101.010002,239.710007,78.150002,80.669998,20.770000,184.699997,52.200001,84.279999,101.010002,240.000000,78.292503,81.430000,21.080000,191.679993,52.660000,85.160004,2107200.0,1652600.0,28694000.0,9775400.0,696300.0,5292800.0,798100.0,500568.0
2020-10-19,11742.037109,107.720001,232.015854,76.730499,78.626236,20.639999,186.960007,51.970001,85.000000,107.720001,238.169998,76.730499,80.610001,20.639999,186.960007,51.970001,85.000000,111.809998,242.899994,79.407501,82.639999,21.270000,191.399994,53.200001,85.519997,106.699997,237.110001,76.400002,80.220001,20.559999,185.910004,51.759998,84.540001,111.300003,240.250000,79.023003,82.000000,20.780001,188.110001,52.410000,85.239998,980900.0,905500.0,32142000.0,8051000.0,636900.0,5454000.0,734300.0,348885.0
